# Setup


In [1]:
import os
# set wd to cancer
os.chdir("/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff")
print(os.getcwd())

/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff


In [2]:
import os
import sys
import math
import copy
import logging
import requests
import zipfile
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from itertools import chain
from typing import Union

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.multioutput import MultiOutputClassifier, MultiOutputRegressor

from pytorch_lightning import LightningModule, Trainer
from omegaconf import DictConfig, OmegaConf
from omegaconf.errors import MissingMandatoryValue

import ray
from ray import tune, ray_constants

import hydra
from hydra import initialize, compose
from hydra.utils import instantiate

from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce

from IPython.core.interactiveshell import InteractiveShell

from src.data import RealDatasetCollection, SyntheticDatasetCollection
from src.models import TimeVaryingCausalModel
from src.models.utils import (
    grad_reverse,
    BRTreatmentOutcomeHead,
    AlphaRise,
    clip_normalize_stabilized_weights,
)
from src.models.utils_lstm import VariationalLSTM
from copy import deepcopy

InteractiveShell.ast_node_interactivity = "all"

In [3]:
# show pytorch version and cuda version
print(torch.__version__)
print(torch.version.cuda)

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of available GPUs: {num_gpus}")
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")
else:
    print("CUDA is not available. No GPUs detected.")

2.0.0+cu118
11.8
Number of available GPUs: 4
GPU 0: NVIDIA A100 80GB PCIe
GPU 1: NVIDIA A100 80GB PCIe
GPU 2: NVIDIA A100 80GB PCIe
GPU 3: NVIDIA A100 80GB PCIe


# MSM


In [ ]:
import logging
import hydra
import torch
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate
from pytorch_lightning import seed_everything

from src.models.utils import FilteringMlFlowLogger
from src.models.msm import MSM


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
torch.set_default_dtype(torch.double)

# Load the configuration file
config_path = '/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/config/cancer_sim_MSM.yaml'
args = OmegaConf.load(config_path)
# @hydra.main(config_name=f'config.yaml', config_path='../config/')


def main(args: DictConfig):
    """
    Training / evaluation script for MSMs
    Args:
        args: arguments of run as DictConfig

    Returns: dict with results (one and nultiple-step-ahead RMSEs)
    """

    results = {}

    # Non-strict access to fields
    OmegaConf.set_struct(args, False)
    OmegaConf.register_new_resolver("sum", lambda x, y: x + y, replace=True)
    logger.info('\n' + OmegaConf.to_yaml(args, resolve=True))

    # Initialisation of data to calculate dim_outcomes, dim_treatments, dim_vitals and dim_static_features
    seed_everything(args.exp.seed)
    dataset_collection = instantiate(args.dataset, _recursive_=True)
    # Only binary multilabel regime possible
    assert args.dataset.treatment_mode == 'multilabel'
    dataset_collection.process_data_multi()
    args.model.dim_outcomes = dataset_collection.train_f.data['outputs'].shape[-1]
    args.model.dim_treatments = dataset_collection.train_f.data['current_treatments'].shape[-1]
    args.model.dim_vitals = dataset_collection.train_f.data[
        'vitals'].shape[-1] if dataset_collection.has_vitals else 0
    args.model.dim_static_features = dataset_collection.train_f.data['static_features'].shape[-1]

    # MlFlow Logger
    if args.exp.logging:
        experiment_name = f'{args.model.name}/{args.dataset.name}'
        mlf_logger = FilteringMlFlowLogger(filter_submodels=MSM.possible_model_types, experiment_name=experiment_name,
                                           tracking_uri=args.exp.mlflow_uri)
    else:
        mlf_logger = None

    # ============================== Nominator (treatment propensity network) ==============================
    propensity_treatment = instantiate(
        args.model.propensity_treatment, args, dataset_collection, _recursive_=False)
    mlf_logger.log_hyperparams(propensity_treatment.hparams)
    propensity_treatment.fit()

    # ============================== Denominator (history propensity network) ==============================
    propensity_history = instantiate(
        args.model.propensity_history, args, dataset_collection, _recursive_=False)
    mlf_logger.log_hyperparams(propensity_history.hparams)
    propensity_history.fit()

    # ============================== Initialisation & Training of Encoder ==============================
    msm_regressor = instantiate(args.model.msm_regressor, args, propensity_treatment, propensity_history, dataset_collection,
                                _recursive_=False)
    mlf_logger.log_hyperparams(msm_regressor.hparams)
    msm_regressor.fit()
    encoder_results = {}

    if hasattr(dataset_collection, 'test_cf_one_step'):  # Test one_step_counterfactual rmse
        test_rmse_orig, test_rmse_all, test_rmse_last = \
            msm_regressor.get_normalised_masked_rmse(
                dataset_collection.test_cf_one_step, one_step_counterfactual=True)
        logger.info(f'Test normalised RMSE (all): {test_rmse_all}; '
                    f'Test normalised RMSE (orig): {test_rmse_orig}; '
                    f'Test normalised RMSE (only counterfactual): {test_rmse_last}')
        encoder_results = {
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig,
            'encoder_test_rmse_last': test_rmse_last
        }
    elif hasattr(dataset_collection, 'test_f'):  # Test factual rmse
        test_rmse_orig, test_rmse_all = msm_regressor.get_normalised_masked_rmse(
            dataset_collection.test_f)
        logger.info(f'Test normalised RMSE (all): {test_rmse_all}; '
                    f'Test normalised RMSE (orig): {test_rmse_orig}.')
        encoder_results = {
            # 'encoder_val_rmse_all': val_rmse_all,
            # 'encoder_val_rmse_orig': val_rmse_orig,
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig
        }

    mlf_logger.log_metrics(encoder_results) if args.exp.logging else None
    results.update(encoder_results)

    test_rmses = {}
    if hasattr(dataset_collection, 'test_cf_treatment_seq'):  # Test n_step_counterfactual rmse
        test_rmses = msm_regressor.get_normalised_n_step_rmses(
            dataset_collection.test_cf_treatment_seq)
    elif hasattr(dataset_collection, 'test_f_multi'):  # Test n_step_factual rmse
        test_rmses = msm_regressor.get_normalised_n_step_rmses(
            dataset_collection.test_f_multi)
    test_rmses = {f'{k+2}-step': v for (k, v) in enumerate(test_rmses)}

    logger.info(f'Test normalised RMSE (n-step prediction): {test_rmses}')
    decoder_results = {('decoder_test_rmse_' + k): v for (k, v) in test_rmses.items()}

    mlf_logger.log_metrics(decoder_results) if args.exp.logging else None
    results.update(decoder_results)

    mlf_logger.experiment.set_terminated(
        mlf_logger.run_id) if args.exp.logging else None

    return results


if __name__ == "__main__":
    main(args)

INFO:__main__:
dataset:
  _target_: src.data.SyntheticCancerDatasetCollection
  name: tumor_generator
  coeff: 4
  chemo_coeff: 4
  radio_coeff: 4
  seed: 100
  num_patients:
    train: 10000
    val: 1000
    test: 100
  window_size: 15
  lag: 0
  max_seq_length: 60
  projection_horizon: 5
  cf_seq_mode: sliding_treatment
  val_batch_size: 512
  treatment_mode: multilabel
model:
  dim_treatments: 4
  dim_vitals: 0
  dim_static_features: 1
  dim_outcomes: 1
  min_length: 1
  lag_features: 1
  name: MSM
  propensity_treatment:
    _target_: src.models.msm.MSMPropensityTreatment
  propensity_history:
    _target_: src.models.msm.MSMPropensityHistory
  msm_regressor:
    _target_: src.models.msm.MSMRegressor
exp:
  unscale_rmse: true
  percentage_rmse: true
  seed: 100
  max_epochs: 100
  gpus:
  - 0
  logging: true
  mlflow_uri: http://127.0.0.1:8081
  alpha: 1.0
  update_alpha: false
  alpha_rate: exp
  balancing: null
  bce_weight: false
  weights_ema: null
  beta: 0.99

Global seed se

INFO:root:Got correlated params for 13793 patients
INFO:root:Simulating beta_c parameters
INFO:root:Randomising outputs
  0%|          | 0/10000 [00:00<?, ?it/s]/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/src/data/cancer_sim/cancer_simulation.py:346: RuntimeWarning: overflow encountered in exp
  if recovery_rvs[i, t] < np.exp(-cancer_volume[i, t] * TUMOUR_CELL_DENSITY):
100%|██████████| 10000/10000 [00:12<00:00, 795.51it/s]
INFO:root:Simulating initial volumes for stage I  with norm params: mu=1.72, sigma=4.7, lb=-0.6221218732608373, ub=-0.023523848418276528
INFO:root:Simulating initial volumes for stage II  with norm params: mu=1.96, sigma=1.63, lb=-1.9410876100159118, ub=0.3711345751297772
INFO:root:Simulating initial volumes for stage IIIA  with norm params: mu=1.91, sigma=9.4, lb=-0.33127370258786554, ub=0.06967546355973796
INFO:root:Simulating initial volumes for stage IIIB  with norm params: mu=2.76, sigma=6.87, lb=-0.5769974969906748, ub=-0.02839165102452

Call to simulate counterfactuals data


100%|██████████| 100/100 [00:02<00:00, 43.78it/s]
INFO:src.data.cancer_sim.dataset:Processing train dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed train data: {'cancer_volume': (10000, 60), 'chemo_dosage': (10000, 60), 'radio_dosage': (10000, 60), 'chemo_application': (10000, 60), 'radio_application': (10000, 60), 'chemo_probabilities': (10000, 60), 'radio_probabilities': (10000, 60), 'sequence_lengths': (10000,), 'death_flags': (10000, 60), 'recovery_flags': (10000, 60), 'patient_types': (10000,), 'prev_treatments': (10000, 59, 2), 'current_treatments': (10000, 59, 2), 'current_covariates': (10000, 59, 2), 'outputs': (10000, 59, 1), 'active_entries': (10000, 59, 1), 'unscaled_outputs': (10000, 59, 1), 'prev_outputs': (10000, 59, 1), 'static_features': (10000, 1)}
INFO:src.data.cancer_sim.dataset:Processing val dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed val data: {'cancer_volume': (1000, 60), 'chemo_dosage': (1000, 60), 'rad

🏃 View run able-shoat-598 at: http://127.0.0.1:8081/#/experiments/997620564669772123/runs/fc7d02f5a5ab46d2961e07eed761e1aa
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/997620564669772123


{'encoder_test_rmse_all': 3.770219048532587,
 'encoder_test_rmse_orig': 2.9484263938352724,
 'encoder_test_rmse_last': 1.305809047323435,
 'decoder_test_rmse_2-step': 2.3833088435242646,
 'decoder_test_rmse_3-step': 2.5292776897711144,
 'decoder_test_rmse_4-step': 2.5056101764583745,
 'decoder_test_rmse_5-step': 2.420787517464479,
 'decoder_test_rmse_6-step': 2.293027285986994}

# RMSN


In [4]:
import logging
import hydra
import torch
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor

from src.models.utils import AlphaRise, FilteringMlFlowLogger
from src.models.rmsn import RMSN

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
torch.set_default_dtype(torch.double)

# Load the configuration file
config_path = '/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/config/cancer_sim_rmsn.yaml'
args = OmegaConf.load(config_path)


def main(args: DictConfig):
    """
    Training / evaluation script for RMSNs
    Args:
        args: arguments of run as DictConfig

    Returns: dict with results (one and nultiple-step-ahead RMSEs)
    """

    results = {}

    # Non-strict access to fields
    OmegaConf.set_struct(args, False)
    OmegaConf.register_new_resolver("sum", lambda x, y: x + y, replace=True)
    logger.info('\n' + OmegaConf.to_yaml(args, resolve=True))

    # Initialisation of data to calculate dim_outcomes, dim_treatments, dim_vitals and dim_static_features
    seed_everything(args.exp.seed)
    dataset_collection = instantiate(args.dataset, _recursive_=True)
    # Only binary multilabel regime possible
    assert args.dataset.treatment_mode == 'multilabel'
    dataset_collection.process_data_encoder()
    args.model.dim_outcomes = dataset_collection.train_f.data['outputs'].shape[-1]
    args.model.dim_treatments = dataset_collection.train_f.data['current_treatments'].shape[-1]
    args.model.dim_vitals = dataset_collection.train_f.data[
        'vitals'].shape[-1] if dataset_collection.has_vitals else 0
    args.model.dim_static_features = dataset_collection.train_f.data['static_features'].shape[-1]

    # Train_callbacks
    prop_treatment_callbacks, propensity_history_callbacks, encoder_callbacks, decoder_callbacks = [], [], [], []

    # MlFlow Logger
    if args.exp.logging:
        experiment_name = f'{args.model.name}/{args.dataset.name}'
        mlf_logger = FilteringMlFlowLogger(filter_submodels=RMSN.possible_model_types, experiment_name=experiment_name,
                                           tracking_uri=args.exp.mlflow_uri)
        encoder_callbacks += [LearningRateMonitor(logging_interval='epoch')]
        decoder_callbacks += [LearningRateMonitor(logging_interval='epoch')]
        prop_treatment_callbacks += [
            LearningRateMonitor(logging_interval='epoch')]
        propensity_history_callbacks += [
            LearningRateMonitor(logging_interval='epoch')]
    else:
        mlf_logger = None

    # ============================== Nominator (treatment propensity network) ==============================
    propensity_treatment = instantiate(
        args.model.propensity_treatment, args, dataset_collection, _recursive_=False)
    if args.model.propensity_treatment.tune_hparams:
        propensity_treatment.finetune(
            resources_per_trial=args.model.propensity_treatment.resources_per_trial)

    propensity_treatment_trainer = Trainer(gpus=eval(str(args.exp.gpus)),
                                           logger=mlf_logger,
                                           max_epochs=args.exp.max_epochs,
                                           callbacks=prop_treatment_callbacks,
                                           gradient_clip_val=args.model.propensity_treatment.max_grad_norm,
                                           terminate_on_nan=True)
    propensity_treatment_trainer.fit(propensity_treatment)

    # Validation BCE
    val_bce_orig, val_bce_all = propensity_treatment.get_masked_bce(
        dataset_collection.val_f)
    logger.info(
        f'Val normalised BCE (all): {val_bce_all}; Val normalised RMSE (orig): {val_bce_orig}')

    # Test BCE
    if hasattr(dataset_collection, 'test_cf_one_step'):  # Test one_step_counterfactual
        test_bce_orig, test_bce_all = propensity_treatment.get_masked_bce(
            dataset_collection.test_cf_one_step)
    elif hasattr(dataset_collection, 'test_f'):  # Test factual
        test_bce_orig, test_bce_all = propensity_treatment.get_masked_bce(
            dataset_collection.test_f)

    logger.info(
        f'Test normalised RMSE (all): {test_bce_orig}; Test normalised RMSE (orig): {test_bce_all}.')
    prop_treatment_results = {
        'propensity_treatment_val_bce_all': val_bce_all,
        'propensity_treatment_val_bce_orig': val_bce_orig,
        'propensity_treatment_test_bce_all': test_bce_all,
        'propensity_treatment_test_bce_orig': test_bce_orig
    }

    mlf_logger.log_metrics(
        prop_treatment_results) if args.exp.logging else None
    results.update(prop_treatment_results)

    # ============================== Denominator (history propensity network) ==============================
    propensity_history = instantiate(
        args.model.propensity_history, args, dataset_collection, _recursive_=False)
    if args.model.propensity_history.tune_hparams:
        propensity_history.finetune(
            resources_per_trial=args.model.propensity_history.resources_per_trial)

    propensity_history_trainer = Trainer(gpus=eval(str(args.exp.gpus)),
                                         logger=mlf_logger,
                                         max_epochs=args.exp.max_epochs,
                                         callbacks=propensity_history_callbacks,
                                         gradient_clip_val=args.model.propensity_history.max_grad_norm,
                                         terminate_on_nan=True)
    propensity_history_trainer.fit(propensity_history)

    # Validation BCE
    val_bce_orig, val_bce_all = propensity_history.get_masked_bce(
        dataset_collection.val_f)
    logger.info(
        f'Val normalised BCE (all): {val_bce_all}; Val normalised RMSE (orig): {val_bce_orig}')

    # Test BCE
    if hasattr(dataset_collection, 'test_cf_one_step'):  # Test one_step_counterfactual
        test_bce_orig, test_bce_all = propensity_history.get_masked_bce(
            dataset_collection.test_cf_one_step)
    elif hasattr(dataset_collection, 'test_f'):  # Test factual
        test_bce_orig, test_bce_all = propensity_history.get_masked_bce(
            dataset_collection.test_f)

    logger.info(
        f'Test normalised BCE (all): {test_bce_orig}; Test normalised BCE (orig): {test_bce_all}.')
    propensity_history_results = {
        'propensity_history_val_bce_all': val_bce_all,
        'propensity_history_val_bce_orig': val_bce_orig,
        'propensity_history_test_bce_all': test_bce_all,
        'propensity_history_test_bce_orig': test_bce_orig
    }

    mlf_logger.log_metrics(
        propensity_history_results) if args.exp.logging else None
    results.update(propensity_history_results)

    # ============================== Initialisation & Training of Encoder ==============================
    encoder = instantiate(args.model.encoder, args, propensity_treatment, propensity_history, dataset_collection,
                          _recursive_=False)
    if args.model.encoder.tune_hparams:
        encoder.finetune(
            resources_per_trial=args.model.encoder.resources_per_trial)

    encoder_trainer = Trainer(gpus=eval(str(args.exp.gpus)),
                              logger=mlf_logger,
                              max_epochs=args.exp.max_epochs,
                              callbacks=encoder_callbacks,
                              gradient_clip_val=args.model.encoder.max_grad_norm,
                              terminate_on_nan=True)
    encoder_trainer.fit(encoder)
    encoder_results = {}

    def test_step(self, batch, batch_idx):
        return self.validation_step(batch, batch_idx)
    encoder.test_step = test_step.__get__(encoder, encoder.__class__)
    # Validation factual rmse
    val_dataloader = DataLoader(
        dataset_collection.val_f, batch_size=args.dataset.val_batch_size, shuffle=False)
    encoder_trainer.test(encoder, dataloaders=val_dataloader)
    val_rmse_orig, val_rmse_all = encoder.get_normalised_masked_rmse(
        dataset_collection.val_f)
    logger.info(
        f'Val normalised RMSE (all): {val_rmse_all}; Val normalised RMSE (orig): {val_rmse_orig}')

    if hasattr(dataset_collection, 'test_cf_one_step'):  # Test one_step_counterfactual rmse
        test_rmse_orig, test_rmse_all, test_rmse_last = encoder.get_normalised_masked_rmse(dataset_collection.test_cf_one_step,
                                                                                           one_step_counterfactual=True)
        logger.info(f'Test normalised RMSE (all): {test_rmse_all}; '
                    f'Test normalised RMSE (orig): {test_rmse_orig}; '
                    f'Test normalised RMSE (only counterfactual): {test_rmse_last}')
        encoder_results = {
            'encoder_val_rmse_all': val_rmse_all,
            'encoder_val_rmse_orig': val_rmse_orig,
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig,
            'encoder_test_rmse_last': test_rmse_last
        }
    elif hasattr(dataset_collection, 'test_f'):  # Test factual rmse
        test_rmse_orig, test_rmse_all = encoder.get_normalised_masked_rmse(
            dataset_collection.test_f)
        logger.info(f'Test normalised RMSE (all): {test_rmse_all}; '
                    f'Test normalised RMSE (orig): {test_rmse_orig}.')
        encoder_results = {
            'encoder_val_rmse_all': val_rmse_all,
            'encoder_val_rmse_orig': val_rmse_orig,
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig
        }

    mlf_logger.log_metrics(encoder_results) if args.exp.logging else None
    results.update(encoder_results)

    # ============================== Initialisation & Training of Decoder ==============================
    if args.model.train_decoder:
        decoder = instantiate(args.model.decoder, args,
                              encoder, dataset_collection, _recursive_=False)

        if args.model.decoder.tune_hparams:
            decoder.finetune(
                resources_per_trial=args.model.decoder.resources_per_trial)

        decoder_trainer = Trainer(gpus=eval(str(args.exp.gpus)),
                                  logger=mlf_logger,
                                  max_epochs=args.exp.max_epochs,
                                  gradient_clip_val=args.model.decoder.max_grad_norm,
                                  callbacks=decoder_callbacks,
                                  terminate_on_nan=True)
        decoder_trainer.fit(decoder)

        def test_step(self, batch, batch_idx):
            return self.validation_step(batch, batch_idx)
        decoder.test_step = test_step.__get__(decoder, decoder.__class__)
        # Validation factual rmse
        val_dataloader = DataLoader(
            dataset_collection.val_f, batch_size=10 * args.dataset.val_batch_size, shuffle=False)
        decoder_trainer.test(decoder, dataloaders=val_dataloader)
        val_rmse_orig, val_rmse_all = decoder.get_normalised_masked_rmse(
            dataset_collection.val_f)
        logger.info(
            f'Val normalised RMSE (all): {val_rmse_all}; Val normalised RMSE (orig): {val_rmse_orig}')

        test_rmses = {}
        if hasattr(dataset_collection, 'test_cf_treatment_seq'):  # Test n_step_counterfactual rmse
            test_rmses = decoder.get_normalised_n_step_rmses(
                dataset_collection.test_cf_treatment_seq)
        elif hasattr(dataset_collection, 'test_f'):  # Test n_step_factual rmse
            test_rmses = decoder.get_normalised_n_step_rmses(
                dataset_collection.test_f)
        test_rmses = {f'{k+2}-step': v for (k, v) in enumerate(test_rmses)}

        logger.info(f'Test normalised RMSE (n-step prediction): {test_rmses}')
        decoder_results = {
            'decoder_val_rmse_all': val_rmse_all,
            'decoder_val_rmse_orig': val_rmse_orig
        }
        decoder_results.update(
            {('decoder_test_rmse_' + k): v for (k, v) in test_rmses.items()})

        mlf_logger.log_metrics(decoder_results) if args.exp.logging else None
        results.update(decoder_results)

        mlf_logger.experiment.set_terminated(
            mlf_logger.run_id) if args.exp.logging else None

    return results


if __name__ == "__main__":
    main(args)

INFO:__main__:
dataset:
  _target_: src.data.SyntheticCancerDatasetCollection
  name: tumor_generator
  coeff: 4
  chemo_coeff: 4
  radio_coeff: 4
  seed: 100
  num_patients:
    train: 10000
    val: 1000
    test: 100
  window_size: 15
  lag: 0
  max_seq_length: 60
  projection_horizon: 5
  cf_seq_mode: sliding_treatment
  val_batch_size: 512
  treatment_mode: multilabel
model:
  name: RMSN
  propensity_treatment:
    _target_: src.models.rmsn.RMSNPropensityNetworkTreatment
    seq_hidden_units: 8
    dropout_rate: 0.2
    num_layer: 1
    batch_size: 64
    max_grad_norm: 2.0
    optimizer:
      optimizer_cls: adam
      learning_rate: 0.01
      weight_decay: 0.0
      lr_scheduler: false
    tune_hparams: false
    tune_range: 50
    resources_per_trial: null
    hparams_grid: null
  propensity_history:
    _target_: src.models.rmsn.RMSNPropensityNetworkHistory
    seq_hidden_units: 16
    dropout_rate: 0.5
    num_layer: 1
    batch_size: 128
    max_grad_norm: 2.0
    optimizer

  0%|          | 0/10000 [00:00<?, ?it/s]/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/src/data/cancer_sim/cancer_simulation.py:346: RuntimeWarning: overflow encountered in exp
  if recovery_rvs[i, t] < np.exp(-cancer_volume[i, t] * TUMOUR_CELL_DENSITY):
100%|██████████| 10000/10000 [00:11<00:00, 890.82it/s]
INFO:root:Simulating initial volumes for stage I  with norm params: mu=1.72, sigma=4.7, lb=-0.6221218732608373, ub=-0.023523848418276528
INFO:root:Simulating initial volumes for stage II  with norm params: mu=1.96, sigma=1.63, lb=-1.9410876100159118, ub=0.3711345751297772
INFO:root:Simulating initial volumes for stage IIIA  with norm params: mu=1.91, sigma=9.4, lb=-0.33127370258786554, ub=0.06967546355973796
INFO:root:Simulating initial volumes for stage IIIB  with norm params: mu=2.76, sigma=6.87, lb=-0.5769974969906748, ub=-0.02839165102452155
INFO:root:Simulating initial volumes for stage IV  with norm params: mu=3.86, sigma=8.82, lb=-0.5741465764541877, ub

Call to simulate counterfactuals data


100%|██████████| 100/100 [00:02<00:00, 48.65it/s]
INFO:src.data.cancer_sim.dataset:Processing train dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed train data: {'cancer_volume': (10000, 60), 'chemo_dosage': (10000, 60), 'radio_dosage': (10000, 60), 'chemo_application': (10000, 60), 'radio_application': (10000, 60), 'chemo_probabilities': (10000, 60), 'radio_probabilities': (10000, 60), 'sequence_lengths': (10000,), 'death_flags': (10000, 60), 'recovery_flags': (10000, 60), 'patient_types': (10000,), 'prev_treatments': (10000, 59, 2), 'current_treatments': (10000, 59, 2), 'current_covariates': (10000, 59, 2), 'outputs': (10000, 59, 1), 'active_entries': (10000, 59, 1), 'unscaled_outputs': (10000, 59, 1), 'prev_outputs': (10000, 59, 1), 'static_features': (10000, 1)}
INFO:src.data.cancer_sim.dataset:Processing val dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed val data: {'cancer_volume': (1000, 60), 'chemo_dosage': (1000, 60), 'rad

Training: 0it [00:00, ?it/s]

...

🏃 View run monumental-lynx-854 at: http://127.0.0.1:8081/#/experiments/649629653407368041/runs/3696161633ff4a2297f8ba44b68a615c
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/649629653407368041


Predicting: 78it [00:00, ?it/s]

INFO:__main__:Test normalised RMSE (all): 1.3741569130772973; Test normalised RMSE (orig): 1.0639230030289093; Test normalised RMSE (only counterfactual): 0.8943832537537814


🏃 View run monumental-lynx-854 at: http://127.0.0.1:8081/#/experiments/649629653407368041/runs/3696161633ff4a2297f8ba44b68a615c
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/649629653407368041


{'propensity_treatment_val_bce_all': 0.7714576178417046,
 'propensity_treatment_val_bce_orig': 0.7715419283315933,
 'propensity_treatment_test_bce_all': 0.772097320947565,
 'propensity_treatment_test_bce_orig': 0.7690497783840289,
 'propensity_history_val_bce_all': 0.7474692722393119,
 'propensity_history_val_bce_orig': 0.7474214821895481,
 'propensity_history_test_bce_all': 0.7810252791743663,
 'propensity_history_test_bce_orig': 0.769577512240865,
 'encoder_val_rmse_all': 0.9644452499106542,
 'encoder_val_rmse_orig': 0.9506212342312618,
 'encoder_test_rmse_all': 1.3741569130772973,
 'encoder_test_rmse_orig': 1.0639230030289093,
 'encoder_test_rmse_last': 0.8943832537537814}

# CT


In [17]:
import logging
import hydra
import torch
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor

from src.models.utils import AlphaRise, FilteringMlFlowLogger

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
torch.set_default_dtype(torch.double)


# Load the configuration file
config_path = '/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/config/cancer_sim_ct.yaml'
args = OmegaConf.load(config_path)


def main(args: DictConfig):
    """
    Training / evaluation script for CT (Causal Transformer)
    Args:
        args: arguments of run as DictConfig

    Returns: dict with results (one and nultiple-step-ahead RMSEs)
    """

    results = {}

    # Non-strict access to fields
    OmegaConf.set_struct(args, False)
    OmegaConf.register_new_resolver("sum", lambda x, y: x + y, replace=True)
    logger.info('\n' + OmegaConf.to_yaml(args, resolve=True))

    # Initialisation of data
    seed_everything(args.exp.seed)
    dataset_collection = instantiate(args.dataset, _recursive_=True)
    dataset_collection.process_data_multi()
    args.model.dim_outcomes = dataset_collection.train_f.data['outputs'].shape[-1]
    args.model.dim_treatments = dataset_collection.train_f.data['current_treatments'].shape[-1]
    args.model.dim_vitals = dataset_collection.train_f.data[
        'vitals'].shape[-1] if dataset_collection.has_vitals else 0
    args.model.dim_static_features = dataset_collection.train_f.data['static_features'].shape[-1]

    # Train_callbacks
    multimodel_callbacks = [AlphaRise(rate=args.exp.alpha_rate)]

    # MlFlow Logger
    if args.exp.logging:
        experiment_name = f'{args.model.name}/{args.dataset.name}'
        mlf_logger = FilteringMlFlowLogger(
            filter_submodels=[], experiment_name=experiment_name, tracking_uri=args.exp.mlflow_uri)
        multimodel_callbacks += [LearningRateMonitor(logging_interval='epoch')]
        artifacts_path = hydra.utils.to_absolute_path(
            mlf_logger.experiment.get_run(mlf_logger.run_id).info.artifact_uri)
    else:
        mlf_logger = None
        artifacts_path = None

    # ============================== Initialisation & Training of multimodel ==============================
    multimodel = instantiate(args.model.multi, args,
                             dataset_collection, _recursive_=False)
    if args.model.multi.tune_hparams:
        multimodel.finetune(
            resources_per_trial=args.model.multi.resources_per_trial)

    multimodel_trainer = Trainer(gpus=eval(str(args.exp.gpus)), logger=mlf_logger, max_epochs=args.exp.max_epochs,
                                 callbacks=multimodel_callbacks, terminate_on_nan=True,
                                 gradient_clip_val=args.model.multi.max_grad_norm)
    multimodel_trainer.fit(multimodel)

    # Validation factual rmse
    val_dataloader = DataLoader(
        dataset_collection.val_f, batch_size=args.dataset.val_batch_size, shuffle=False)
    multimodel.test_dataloader = lambda: val_dataloader
    multimodel_trainer.test(multimodel, dataloaders=val_dataloader)
    # multimodel.visualize(dataset_collection.val_f, index=0, artifacts_path=artifacts_path)
    val_rmse_orig, val_rmse_all = multimodel.get_normalised_masked_rmse(
        dataset_collection.val_f)
    logger.info(
        f'Val normalised RMSE (all): {val_rmse_all}; Val normalised RMSE (orig): {val_rmse_orig}')

    encoder_results = {}
    if hasattr(dataset_collection, 'test_cf_one_step'):  # Test one_step_counterfactual rmse
        test_rmse_orig, test_rmse_all, test_rmse_last = multimodel.get_normalised_masked_rmse(dataset_collection.test_cf_one_step,
                                                                                              one_step_counterfactual=True)
        logger.info(f'Test normalised RMSE (all): {test_rmse_all}; '
                    f'Test normalised RMSE (orig): {test_rmse_orig}; '
                    f'Test normalised RMSE (only counterfactual): {test_rmse_last}')
        encoder_results = {
            'encoder_val_rmse_all': val_rmse_all,
            'encoder_val_rmse_orig': val_rmse_orig,
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig,
            'encoder_test_rmse_last': test_rmse_last
        }
    elif hasattr(dataset_collection, 'test_f'):  # Test factual rmse
        test_rmse_orig, test_rmse_all = multimodel.get_normalised_masked_rmse(
            dataset_collection.test_f)
        logger.info(f'Test normalised RMSE (all): {test_rmse_all}; '
                    f'Test normalised RMSE (orig): {test_rmse_orig}.')
        encoder_results = {
            'encoder_val_rmse_all': val_rmse_all,
            'encoder_val_rmse_orig': val_rmse_orig,
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig
        }

    mlf_logger.log_metrics(encoder_results) if args.exp.logging else None
    results.update(encoder_results)

    test_rmses = {}
    if hasattr(dataset_collection, 'test_cf_treatment_seq'):  # Test n_step_counterfactual rmse
        test_rmses = multimodel.get_normalised_n_step_rmses(
            dataset_collection.test_cf_treatment_seq)
    elif hasattr(dataset_collection, 'test_f_multi'):  # Test n_step_factual rmse
        test_rmses = multimodel.get_normalised_n_step_rmses(
            dataset_collection.test_f_multi)
    test_rmses = {f'{k+2}-step': v for (k, v) in enumerate(test_rmses)}

    logger.info(f'Test normalised RMSE (n-step prediction): {test_rmses}')
    decoder_results = {
        'decoder_val_rmse_all': val_rmse_all,
        'decoder_val_rmse_orig': val_rmse_orig
    }
    decoder_results.update(
        {('decoder_test_rmse_' + k): v for (k, v) in test_rmses.items()})

    mlf_logger.log_metrics(decoder_results) if args.exp.logging else None
    results.update(decoder_results)

    mlf_logger.experiment.set_terminated(
        mlf_logger.run_id) if args.exp.logging else None

    return results


if __name__ == "__main__":
    main(args)

INFO:__main__:
dataset:
  _target_: src.data.SyntheticCancerDatasetCollection
  name: tumor_generator
  coeff: 4
  chemo_coeff: 4
  radio_coeff: 4
  seed: 100
  num_patients:
    train: 10000
    val: 1000
    test: 100
  window_size: 15
  lag: 0
  max_seq_length: 60
  projection_horizon: 5
  cf_seq_mode: sliding_treatment
  val_batch_size: 512
  treatment_mode: multiclass
model:
  name: CT_DomainConfusion
  dim_treatments: 4
  dim_vitals: 0
  dim_static_features: 1
  dim_outcomes: 1
  multi:
    tune_hparams: false
    batch_size: 128
    seq_hidden_units: 12
    br_size: 16
    fc_hidden_units: 48
    dropout_rate: 0.1
    _target_: src.models.ct.CT
    max_seq_length: 65
    num_layer: 1
    num_heads: 2
    max_grad_norm: 1
    attn_dropout: true
    disable_cross_attention: false
    isolate_subnetwork: _
    self_positional_encoding:
      absolute: false
      trainable: true
      max_relative_position: 15
    optimizer:
      optimizer_cls: adam
      learning_rate: 0.01
     

/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/src/data/cancer_sim/cancer_simulation.py:346: RuntimeWarning: overflow encountered in exp
  if recovery_rvs[i, t] < np.exp(-cancer_volume[i, t] * TUMOUR_CELL_DENSITY):
100%|██████████| 10000/10000 [00:14<00:00, 670.47it/s]
INFO:root:Simulating initial volumes for stage I  with norm params: mu=1.72, sigma=4.7, lb=-0.6221218732608373, ub=-0.023523848418276528
INFO:root:Simulating initial volumes for stage II  with norm params: mu=1.96, sigma=1.63, lb=-1.9410876100159118, ub=0.3711345751297772
INFO:root:Simulating initial volumes for stage IIIA  with norm params: mu=1.91, sigma=9.4, lb=-0.33127370258786554, ub=0.06967546355973796
INFO:root:Simulating initial volumes for stage IIIB  with norm params: mu=2.76, sigma=6.87, lb=-0.5769974969906748, ub=-0.02839165102452155
INFO:root:Simulating initial volumes for stage IV  with norm params: mu=3.86, sigma=8.82, lb=-0.5741465764541877, ub=-0.14683113860980307
INFO:root:Got corre

Call to simulate counterfactuals data


100%|██████████| 100/100 [00:02<00:00, 37.30it/s]
INFO:src.data.cancer_sim.dataset:Processing train dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed train data: {'cancer_volume': (10000, 60), 'chemo_dosage': (10000, 60), 'radio_dosage': (10000, 60), 'chemo_application': (10000, 60), 'radio_application': (10000, 60), 'chemo_probabilities': (10000, 60), 'radio_probabilities': (10000, 60), 'sequence_lengths': (10000,), 'death_flags': (10000, 60), 'recovery_flags': (10000, 60), 'patient_types': (10000,), 'prev_treatments': (10000, 59, 4), 'current_treatments': (10000, 59, 4), 'current_covariates': (10000, 59, 2), 'outputs': (10000, 59, 1), 'active_entries': (10000, 59, 1), 'unscaled_outputs': (10000, 59, 1), 'prev_outputs': (10000, 59, 1), 'static_features': (10000, 1)}
INFO:src.data.cancer_sim.dataset:Processing val dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed val data: {'cancer_volume': (1000, 60), 'chemo_dosage': (1000, 60), 'rad

Training: 0it [00:00, ?it/s]

...

Predicting: 78it [00:00, ?it/s]

🏃 View run smiling-boar-846 at: http://127.0.0.1:8081/#/experiments/675413508655406484/runs/a0dbafb131bc4660ae59e5c9721b931c
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/675413508655406484


INFO:__main__:Test normalised RMSE (n-step prediction): {'2-step': 0.8045389723075197, '3-step': 0.8697235020318155, '4-step': 0.9632260967138906, '5-step': 1.036357108944081, '6-step': 1.145490341173136}


🏃 View run smiling-boar-846 at: http://127.0.0.1:8081/#/experiments/675413508655406484/runs/a0dbafb131bc4660ae59e5c9721b931c
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/675413508655406484


{'encoder_val_rmse_all': 0.6277229108583364,
 'encoder_val_rmse_orig': 0.6194985587934652,
 'encoder_test_rmse_all': 1.2069718143967407,
 'encoder_test_rmse_orig': 0.9752487229288143,
 'encoder_test_rmse_last': 0.9303795568607486,
 'decoder_val_rmse_all': 0.6277229108583364,
 'decoder_val_rmse_orig': 0.6194985587934652,
 'decoder_test_rmse_2-step': 0.8045389723075197,
 'decoder_test_rmse_3-step': 0.8697235020318155,
 'decoder_test_rmse_4-step': 0.9632260967138906,
 'decoder_test_rmse_5-step': 1.036357108944081,
 'decoder_test_rmse_6-step': 1.145490341173136}

# ENC-DEC (CRN)

In [23]:
import logging
import hydra
import torch
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor

from src.models.utils import AlphaRise, FilteringMlFlowLogger

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
torch.set_default_dtype(torch.double)

# Load the configuration file
config_path = '/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/config/cancer_sim_crn.yaml'
args = OmegaConf.load(config_path)


def main(args: DictConfig):
    """
    Training / evaluation script for models with encoder-decoder structure: CRN, EDCT (Encoder-Decoder Causal Transformer)
    Args:
        args: arguments of run as DictConfig

    Returns: dict with results (one and nultiple-step-ahead RMSEs)
    """

    results = {}

    # Non-strict access to fields
    OmegaConf.set_struct(args, False)
    OmegaConf.register_new_resolver("sum", lambda x, y: x + y, replace=True)
    logger.info('\n' + OmegaConf.to_yaml(args, resolve=True))

    # Initialisation of data
    seed_everything(args.exp.seed)
    dataset_collection = instantiate(args.dataset, _recursive_=True)
    dataset_collection.process_data_encoder()
    args.model.dim_outcomes = dataset_collection.train_f.data['outputs'].shape[-1]
    args.model.dim_treatments = dataset_collection.train_f.data['current_treatments'].shape[-1]
    args.model.dim_vitals = dataset_collection.train_f.data[
        'vitals'].shape[-1] if dataset_collection.has_vitals else 0
    args.model.dim_static_features = dataset_collection.train_f.data['static_features'].shape[-1]

    # Train_callbacks
    encoder_callbacks, decoder_callbacks = [AlphaRise(rate=args.exp.alpha_rate)], [
        AlphaRise(rate=args.exp.alpha_rate)]

    # MlFlow Logger
    if args.exp.logging:
        experiment_name = f'{args.model.name}/{args.dataset.name}'
        mlf_logger = FilteringMlFlowLogger(filter_submodels=['encoder', 'decoder'],
                                           experiment_name=experiment_name, tracking_uri=args.exp.mlflow_uri)
        encoder_callbacks += [LearningRateMonitor(logging_interval='epoch')]
        decoder_callbacks += [LearningRateMonitor(logging_interval='epoch')]
        artifacts_path = hydra.utils.to_absolute_path(
            mlf_logger.experiment.get_run(mlf_logger.run_id).info.artifact_uri)
    else:
        mlf_logger = None
        artifacts_path = None

    # ============================== Initialisation & Training of encoder ==============================
    encoder = instantiate(args.model.encoder, args,
                          dataset_collection, _recursive_=False)
    if args.model.encoder.tune_hparams:
        encoder.finetune(
            resources_per_trial=args.model.encoder.resources_per_trial)

    encoder_trainer = Trainer(gpus=eval(str(args.exp.gpus)), logger=mlf_logger, max_epochs=args.exp.max_epochs,
                              callbacks=encoder_callbacks, terminate_on_nan=True)
    encoder_trainer.fit(encoder)
    encoder_results = {}

    # Validation factual rmse
    val_dataloader = DataLoader(
        dataset_collection.val_f, batch_size=args.dataset.val_batch_size, shuffle=False)
    encoder.test_dataloader = lambda: val_dataloader
    encoder_trainer.test(encoder, dataloaders=val_dataloader)
    val_rmse_orig, val_rmse_all = encoder.get_normalised_masked_rmse(
        dataset_collection.val_f)
    logger.info(
        f'Val normalised RMSE (all): {val_rmse_all}; Val normalised RMSE (orig): {val_rmse_orig}')

    if hasattr(dataset_collection, 'test_cf_one_step'):  # Test one_step_counterfactual rmse
        test_rmse_orig, test_rmse_all, test_rmse_last = encoder.get_normalised_masked_rmse(dataset_collection.test_cf_one_step,
                                                                                           one_step_counterfactual=True)
        logger.info(f'Test normalised RMSE (all): {test_rmse_all}; '
                    f'Test normalised RMSE (orig): {test_rmse_orig}; '
                    f'Test normalised RMSE (only counterfactual): {test_rmse_last}')
        encoder_results = {
            'encoder_val_rmse_all': val_rmse_all,
            'encoder_val_rmse_orig': val_rmse_orig,
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig,
            'encoder_test_rmse_last': test_rmse_last
        }
    elif hasattr(dataset_collection, 'test_f'):  # Test factual rmse
        test_rmse_orig, test_rmse_all = encoder.get_normalised_masked_rmse(
            dataset_collection.test_f)
        logger.info(f'Test normalised RMSE (all): {test_rmse_all}; '
                    f'Test normalised RMSE (orig): {test_rmse_orig}.')
        encoder_results = {
            'encoder_val_rmse_all': val_rmse_all,
            'encoder_val_rmse_orig': val_rmse_orig,
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig
        }

    mlf_logger.log_metrics(encoder_results) if args.exp.logging else None
    results.update(encoder_results)

    # ============================== Initialisation & Training of decoder ==============================
    if args.model.train_decoder:
        if args.exp.logging:
            mlf_logger.filter_submodel = 'encoder'  # Disable Logging to Mlflow
        decoder = instantiate(args.model.decoder, args,
                              encoder, dataset_collection, _recursive_=False)

        if args.model.decoder.tune_hparams:
            decoder.finetune(
                resources_per_trial=args.model.decoder.resources_per_trial)

        decoder_trainer = Trainer(gpus=eval(str(args.exp.gpus)), logger=mlf_logger, max_epochs=args.exp.max_epochs,
                                  callbacks=decoder_callbacks, terminate_on_nan=True)
        decoder_trainer.fit(decoder)

        # Validation factual rmse
        val_dataloader = DataLoader(
            dataset_collection.val_f, batch_size=10 * args.dataset.val_batch_size, shuffle=False)
        decoder.test_dataloader = lambda: val_dataloader
        decoder_trainer.test(decoder, dataloaders=val_dataloader)
        # decoder.visualize(dataset_collection.val_f, index=20, artifacts_path=artifacts_path)
        val_rmse_orig, val_rmse_all = decoder.get_normalised_masked_rmse(
            dataset_collection.val_f)
        logger.info(
            f'Val normalised RMSE (all): {val_rmse_all}; Val normalised RMSE (orig): {val_rmse_orig}')

        test_rmses = {}
        if hasattr(dataset_collection, 'test_cf_treatment_seq'):  # Test n_step_counterfactual rmse
            test_rmses = decoder.get_normalised_n_step_rmses(
                dataset_collection.test_cf_treatment_seq)
        elif hasattr(dataset_collection, 'test_f'):  # Test n_step_factual rmse
            test_rmses = decoder.get_normalised_n_step_rmses(
                dataset_collection.test_f)
        test_rmses = {f'{k+2}-step': v for (k, v) in enumerate(test_rmses)}

        logger.info(f'Test normalised RMSE (n-step prediction): {test_rmses}')
        decoder_results = {
            'decoder_val_rmse_all': val_rmse_all,
            'decoder_val_rmse_orig': val_rmse_orig
        }
        decoder_results.update(
            {('decoder_test_rmse_' + k): v for (k, v) in test_rmses.items()})

        mlf_logger.log_metrics(decoder_results) if args.exp.logging else None
        results.update(decoder_results)

    return results


if __name__ == "__main__":
    main(args)

INFO:__main__:
dataset:
  _target_: src.data.SyntheticCancerDatasetCollection
  name: tumor_generator
  coeff: 4
  chemo_coeff: 4
  radio_coeff: 4
  seed: 100
  num_patients:
    train: 10000
    val: 1000
    test: 100
  window_size: 15
  lag: 0
  max_seq_length: 60
  projection_horizon: 5
  cf_seq_mode: sliding_treatment
  val_batch_size: 512
  treatment_mode: multiclass
model:
  name: CRN
  encoder:
    _target_: src.models.crn.CRNEncoder
    seq_hidden_units: 24
    br_size: 6
    fc_hidden_units: 18
    dropout_rate: 0.2
    num_layer: 1
    batch_size: 64
    optimizer:
      optimizer_cls: adam
      learning_rate: 0.01
      weight_decay: 0.0
      lr_scheduler: false
    tune_hparams: false
    tune_range: 50
    hparams_grid: null
    resources_per_trial: null
  train_decoder: true
  decoder:
    _target_: src.models.crn.CRNDecoder
    seq_hidden_units: 24
    br_size: 3
    fc_hidden_units: 9
    dropout_rate: 0.2
    num_layer: 1
    batch_size: 512
    optimizer:
      opt

/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/src/data/cancer_sim/cancer_simulation.py:346: RuntimeWarning: overflow encountered in exp
  if recovery_rvs[i, t] < np.exp(-cancer_volume[i, t] * TUMOUR_CELL_DENSITY):
100%|██████████| 10000/10000 [00:13<00:00, 736.24it/s]
INFO:root:Simulating initial volumes for stage I  with norm params: mu=1.72, sigma=4.7, lb=-0.6221218732608373, ub=-0.023523848418276528
INFO:root:Simulating initial volumes for stage II  with norm params: mu=1.96, sigma=1.63, lb=-1.9410876100159118, ub=0.3711345751297772
INFO:root:Simulating initial volumes for stage IIIA  with norm params: mu=1.91, sigma=9.4, lb=-0.33127370258786554, ub=0.06967546355973796
INFO:root:Simulating initial volumes for stage IIIB  with norm params: mu=2.76, sigma=6.87, lb=-0.5769974969906748, ub=-0.02839165102452155
INFO:root:Simulating initial volumes for stage IV  with norm params: mu=3.86, sigma=8.82, lb=-0.5741465764541877, ub=-0.14683113860980307
INFO:root:Got corre

Call to simulate counterfactuals data


100%|██████████| 100/100 [00:02<00:00, 39.56it/s]
INFO:src.data.cancer_sim.dataset:Processing train dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed train data: {'cancer_volume': (10000, 60), 'chemo_dosage': (10000, 60), 'radio_dosage': (10000, 60), 'chemo_application': (10000, 60), 'radio_application': (10000, 60), 'chemo_probabilities': (10000, 60), 'radio_probabilities': (10000, 60), 'sequence_lengths': (10000,), 'death_flags': (10000, 60), 'recovery_flags': (10000, 60), 'patient_types': (10000,), 'prev_treatments': (10000, 59, 4), 'current_treatments': (10000, 59, 4), 'current_covariates': (10000, 59, 2), 'outputs': (10000, 59, 1), 'active_entries': (10000, 59, 1), 'unscaled_outputs': (10000, 59, 1), 'prev_outputs': (10000, 59, 1), 'static_features': (10000, 1)}
INFO:src.data.cancer_sim.dataset:Processing val dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed val data: {'cancer_volume': (1000, 60), 'chemo_dosage': (1000, 60), 'rad

Training: 0it [00:00, ?it/s]

...

🏃 View run beautiful-deer-740 at: http://127.0.0.1:8081/#/experiments/964898661885218207/runs/701a23f92c704249bc352afcace92483
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/964898661885218207


Predicting: 521it [00:00, ?it/s]

INFO:__main__:Test normalised RMSE (n-step prediction): {'2-step': 1.0503509926371306, '3-step': 1.3573320421704556, '4-step': 1.5523674763864073, '5-step': 1.6602506639432586, '6-step': 1.688794927133173}


🏃 View run beautiful-deer-740 at: http://127.0.0.1:8081/#/experiments/964898661885218207/runs/701a23f92c704249bc352afcace92483
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/964898661885218207


{'encoder_val_rmse_all': 0.6896331577905638,
 'encoder_val_rmse_orig': 0.6804393706782758,
 'encoder_test_rmse_all': 1.176439998940352,
 'encoder_test_rmse_orig': 0.9107615559797291,
 'encoder_test_rmse_last': 0.47857569581446185,
 'decoder_val_rmse_all': 0.404399824899808,
 'decoder_val_rmse_orig': 0.40439982489980897,
 'decoder_test_rmse_2-step': 1.0503509926371306,
 'decoder_test_rmse_3-step': 1.3573320421704556,
 'decoder_test_rmse_4-step': 1.5523674763864073,
 'decoder_test_rmse_5-step': 1.6602506639432586,
 'decoder_test_rmse_6-step': 1.688794927133173}

# G-Net

In [4]:
import logging
import hydra
import torch
from omegaconf import DictConfig, OmegaConf
from hydra.utils import instantiate
from torch.utils.data import DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor

from src.models.utils import AlphaRise, FilteringMlFlowLogger

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
torch.set_default_dtype(torch.double)


# Load the configuration file
config_path = '/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/config/cancer_sim_gnet.yaml'
args = OmegaConf.load(config_path)


def main(args: DictConfig):
    """
    Training / evaluation script for G-Net
    Args:
        args: arguments of run as DictConfig

    Returns: dict with results (one and nultiple-step-ahead RMSEs)
    """

    results = {}

    # Non-strict access to fields
    OmegaConf.set_struct(args, False)
    OmegaConf.register_new_resolver("sum", lambda x, y: x + y, replace=True)
    logger.info('\n' + OmegaConf.to_yaml(args, resolve=True))

    # Initialisation of data
    seed_everything(args.exp.seed)
    dataset_collection = instantiate(args.dataset, _recursive_=True)
    dataset_collection.process_data_multi()
    args.model.dim_outcomes = dataset_collection.train_f.data['outputs'].shape[-1]
    args.model.dim_treatments = dataset_collection.train_f.data['current_treatments'].shape[-1]
    args.model.dim_vitals = dataset_collection.train_f.data[
        'vitals'].shape[-1] if dataset_collection.has_vitals else 0
    args.model.dim_static_features = dataset_collection.train_f.data['static_features'].shape[-1]

    # Conditional networks outputs are of uniform dim between (dim_outcomes + dim_vitals)
    args.model.g_net.comp_sizes = \
        [(args.model.dim_outcomes + args.model.dim_vitals) //
         args.model.g_net.num_comp] * args.model.g_net.num_comp

    # MlFlow Logger
    if args.exp.logging:
        experiment_name = f'{args.model.name}/{args.dataset.name}'
        mlf_logger = FilteringMlFlowLogger(
            filter_submodels=[], experiment_name=experiment_name, tracking_uri=args.exp.mlflow_uri)
        model_callbacks = [LearningRateMonitor(logging_interval='epoch')]
        artifacts_path = hydra.utils.to_absolute_path(
            mlf_logger.experiment.get_run(mlf_logger.run_id).info.artifact_uri)
    else:
        mlf_logger = None
        artifacts_path = None
        model_callbacks = []

    # ============================== Initialisation & Training of G-Net ==============================
    model = instantiate(args.model.g_net, args,
                        dataset_collection, _recursive_=False)
    if args.model.g_net.tune_hparams:
        model.finetune(
            resources_per_trial=args.model.g_net.resources_per_trial)

    trainer = Trainer(gpus=eval(str(args.exp.gpus)), logger=mlf_logger, max_epochs=args.exp.max_epochs, callbacks=model_callbacks,
                      terminate_on_nan=True)
    trainer.fit(model)

    # Validation factual rmse
    val_rmse_orig, val_rmse_all = model.get_normalised_masked_rmse(
        dataset_collection.val_f)
    logger.info(
        f'Val normalised RMSE (all): {val_rmse_all}; Val normalised RMSE (orig): {val_rmse_orig}')

    encoder_results = {}
    if hasattr(dataset_collection, 'test_cf_one_step'):  # Test one_step_counterfactual rmse
        test_rmse_orig, test_rmse_all, test_rmse_last = model.get_normalised_masked_rmse(dataset_collection.test_cf_one_step,
                                                                                         one_step_counterfactual=True)
        logger.info(f'Test normalised RMSE (all): {test_rmse_all}; '
                    f'Test normalised RMSE (orig): {test_rmse_orig}; '
                    f'Test normalised RMSE (only counterfactual): {test_rmse_last}')
        encoder_results = {
            'encoder_val_rmse_all': val_rmse_all,
            'encoder_val_rmse_orig': val_rmse_orig,
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig,
            'encoder_test_rmse_last': test_rmse_last
        }
    elif hasattr(dataset_collection, 'test_f'):  # Test factual rmse
        test_rmse_orig, test_rmse_all = model.get_normalised_masked_rmse(
            dataset_collection.test_f)
        logger.info(
            f'Test normalised RMSE (all): {test_rmse_all}; Test normalised RMSE (orig): {test_rmse_orig}.')
        encoder_results = {
            'encoder_val_rmse_all': val_rmse_all,
            'encoder_val_rmse_orig': val_rmse_orig,
            'encoder_test_rmse_all': test_rmse_all,
            'encoder_test_rmse_orig': test_rmse_orig
        }

    mlf_logger.log_metrics(encoder_results) if args.exp.logging else None
    results.update(encoder_results)

    test_rmses = {}
    if hasattr(dataset_collection, 'test_cf_treatment_seq_mc'):  # Test n_step_counterfactual rmse
        test_rmses = model.get_normalised_n_step_rmses(dataset_collection.test_cf_treatment_seq,
                                                       dataset_collection.test_cf_treatment_seq_mc)
    elif hasattr(dataset_collection, 'test_f_mc'):  # Test n_step_factual rmse
        test_rmses = model.get_normalised_n_step_rmses(
            dataset_collection.test_f, dataset_collection.test_f_mc)

    test_rmses = {f'{k+2}-step': v for (k, v) in enumerate(test_rmses)}

    logger.info(f'Test normalised RMSE (n-step prediction): {test_rmses}')
    decoder_results = {
        'decoder_val_rmse_all': val_rmse_all,
        'decoder_val_rmse_orig': val_rmse_orig
    }
    decoder_results.update(
        {('decoder_test_rmse_' + k): v for (k, v) in test_rmses.items()})

    mlf_logger.log_metrics(decoder_results) if args.exp.logging else None
    results.update(decoder_results)
    mlf_logger.experiment.set_terminated(
        mlf_logger.run_id) if args.exp.logging else None

    return results


if __name__ == "__main__":
    main(args)

INFO:__main__:
dataset:
  _target_: src.data.SyntheticCancerDatasetCollection
  name: tumor_generator
  coeff: 4
  chemo_coeff: 4
  radio_coeff: 4
  seed: 100
  num_patients:
    train: 10000
    val: 1000
    test: 100
  window_size: 15
  lag: 0
  max_seq_length: 60
  projection_horizon: 5
  cf_seq_mode: sliding_treatment
  val_batch_size: 4096
  treatment_mode: multiclass
  holdout_ratio: 0.1
model:
  name: G-Net
  g_net:
    _target_: src.models.gnet.GNet
    seq_hidden_units: 24
    num_layer: 1
    r_size: 18
    fc_hidden_units: 72
    num_comp: 1
    mc_samples: 50
    fit_vitals: true
    comp_sizes: []
    batch_size: 256
    dropout_rate: 0.1
    optimizer:
      optimizer_cls: adam
      learning_rate: 0.01
      weight_decay: 0.0
      lr_scheduler: false
    tune_hparams: false
    tune_range: 50
    hparams_grid: null
    resources_per_trial: null
exp:
  update_alpha: false
  unscale_rmse: true
  percentage_rmse: true
  seed: 100
  weights_ema: true
  gpus:
  - 0
  loggin

/work/postresearch/Shared/Researchers/Farbod/cancer/code/CausalDiff/src/data/cancer_sim/cancer_simulation.py:346: RuntimeWarning: overflow encountered in exp
  if recovery_rvs[i, t] < np.exp(-cancer_volume[i, t] * TUMOUR_CELL_DENSITY):
100%|██████████| 10000/10000 [00:11<00:00, 893.50it/s]
INFO:root:Simulating initial volumes for stage I  with norm params: mu=1.72, sigma=4.7, lb=-0.6221218732608373, ub=-0.023523848418276528
INFO:root:Simulating initial volumes for stage II  with norm params: mu=1.96, sigma=1.63, lb=-1.9410876100159118, ub=0.3711345751297772
INFO:root:Simulating initial volumes for stage IIIA  with norm params: mu=1.91, sigma=9.4, lb=-0.33127370258786554, ub=0.06967546355973796
INFO:root:Simulating initial volumes for stage IIIB  with norm params: mu=2.76, sigma=6.87, lb=-0.5769974969906748, ub=-0.02839165102452155
INFO:root:Simulating initial volumes for stage IV  with norm params: mu=3.86, sigma=8.82, lb=-0.5741465764541877, ub=-0.14683113860980307
INFO:root:Got corre

Call to simulate counterfactuals data


100%|██████████| 100/100 [00:02<00:00, 48.74it/s]
INFO:src.data.cancer_sim.dataset:Processing train dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed train data: {'cancer_volume': (10000, 60), 'chemo_dosage': (10000, 60), 'radio_dosage': (10000, 60), 'chemo_application': (10000, 60), 'radio_application': (10000, 60), 'chemo_probabilities': (10000, 60), 'radio_probabilities': (10000, 60), 'sequence_lengths': (10000,), 'death_flags': (10000, 60), 'recovery_flags': (10000, 60), 'patient_types': (10000,), 'prev_treatments': (10000, 59, 4), 'current_treatments': (10000, 59, 4), 'current_covariates': (10000, 59, 2), 'outputs': (10000, 59, 1), 'active_entries': (10000, 59, 1), 'unscaled_outputs': (10000, 59, 1), 'prev_outputs': (10000, 59, 1), 'static_features': (10000, 1)}
INFO:src.data.cancer_sim.dataset:Processing val dataset before training
INFO:src.data.cancer_sim.dataset:Shape of processed val data: {'cancer_volume': (1000, 60), 'chemo_dosage': (1000, 60), 'rad

Training: 0it [00:00, ?it/s]

...

🏃 View run delicate-seal-790 at: http://127.0.0.1:8081/#/experiments/887994607810552918/runs/fb94cd8dc7804c5b9f8a26fc1c97f679
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/887994607810552918


Predicting: 35it [00:00, ?it/s]

INFO:__main__:Test normalised RMSE (n-step prediction): {'2-step': 1.4451800245329933, '3-step': 1.8008652533312692, '4-step': 2.0004625959842484, '5-step': 2.1730871523048516, '6-step': 2.343778518295649}


🏃 View run delicate-seal-790 at: http://127.0.0.1:8081/#/experiments/887994607810552918/runs/fb94cd8dc7804c5b9f8a26fc1c97f679
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/887994607810552918
🏃 View run delicate-seal-790 at: http://127.0.0.1:8081/#/experiments/887994607810552918/runs/fb94cd8dc7804c5b9f8a26fc1c97f679
🧪 View experiment at: http://127.0.0.1:8081/#/experiments/887994607810552918


{'encoder_val_rmse_all': 0.6958152216516413,
 'encoder_val_rmse_orig': 0.6864977016771162,
 'encoder_test_rmse_all': 1.0475353371382192,
 'encoder_test_rmse_orig': 0.818291905494028,
 'encoder_test_rmse_last': 0.7632298904565646,
 'decoder_val_rmse_all': 0.6958152216516413,
 'decoder_val_rmse_orig': 0.6864977016771162,
 'decoder_test_rmse_2-step': 1.4451800245329933,
 'decoder_test_rmse_3-step': 1.8008652533312692,
 'decoder_test_rmse_4-step': 2.0004625959842484,
 'decoder_test_rmse_5-step': 2.1730871523048516,
 'decoder_test_rmse_6-step': 2.343778518295649}